# Scrape human made images

This code is designed to scrape images from Google search results. It utilizes Selenium and Chromedriver to automate the process of visiting search result pages, scrolling down to load more images, and extracting the image URLs. The images are then downloaded and saved into specific category folders.

In [ ]:
from multiprocessing.sharedctypes import Value
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image
import time
import os
import urllib

HUMAN = 'C:\\Users\\Karl Marie Yazigi\\Desktop\\HUMAN\\'

# Download the driver from the chromedriver website for the relevant OS, e.g., MAC, Windows, Debian, etc.
PATH = 'C:\\Users\\Karl Marie Yazigi\\Desktop\\chromedriver_win32\\chromedriver'

# URLs to visit
urls = ['https://www.google.com/search?q=art+graphic+design&tbm=isch&ved=2ahUKEwisiM6s2pr-AhXPGTQIHXYSB84Q2-cCegQIABAA&oq=art+graphic+design&gs_lcp=CgNpbWcQAzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1DxAliJCmDFDmgAcAB4AIABb4gB1AWSAQM3LjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=bZYxZOydC8-z0PEP9qSc8Aw&bih=609&biw=1280',
       'https://www.google.com/search?q=anime+art&tbm=isch&ved=2ahUKEwjdz9iv2pr-AhUyAzQIHXdzAjMQ2-cCegQIABAA&oq=anime+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoHCAAQigUQQzoECCMQJ1DjBFinDWDoD2gAcAB4AIABbIgBvgeSAQM3LjOYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=c5YxZJ3KJ7KG0PEP9-aJmAM&bih=609&biw=1280',
       'https://www.google.com/search?q=people+outside&tbm=isch&ved=2ahUKEwid5LC33pr-AhX0HzQIHWLOAkMQ2-cCegQIABAA&oq=people+outside&gs_lcp=CgNpbWcQAzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6BwgAEIoFEEM6CggAEIoFELEDEEM6CAgAEIAEELEDOggIABCxAxCDAToGCAAQCBAeUMsIWNkrYJUsaARwAHgAgAFtiAHPDJIBBDE2LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=tZoxZN21H_S_0PEP4pyLmAQ&bih=609&biw=1280',
       'https://www.google.com/search?q=digital+art&tbm=isch&ved=2ahUKEwjejJ7B3pr-AhWLATQIHQqgCDMQ2-cCegQIABAA&oq=digital+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEOgQIIxAnOggIABCABBCxAzoICAAQsQMQgwFQlgVY7Q9guRRoAHAAeACAAWmIAdgIkgEEMTAuMpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=ypoxZN7_CouD0PEPisCimAM&bih=609&biw=1280',
       'https://www.google.com/search?q=digital+art&tbm=isch&chips=q:digital+art,g_1:portrait:rxaMKhJ3aNU%3D&hl=en&sa=X&ved=2ahUKEwiazeWH35r-AhUTJ30KHXJpCOoQ4lYoBHoECAEQLg&biw=1263&bih=609',
       'https://www.google.com/search?q=digital+art&tbm=isch&chips=q:digital+art,g_1:drawing:wv9WUjQunSk%3D&hl=en&sa=X&ved=2ahUKEwiazeWH35r-AhUTJ30KHXJpCOoQ4lYoBnoECAEQMg&biw=1263&bih=609',
       'https://www.google.com/search?q=abstract+art&tbm=isch&ved=2ahUKEwixmvia35r-AhXMBTQIHWKLBCkQ2-cCegQIABAA&oq=abstract+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1CwBFjxD2CPEmgAcAB4AIABaYgBowmSAQQxMS4ymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=hpsxZLG_EsyL0PEP4paSyAI&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=paintings&tbm=isch&ved=2ahUKEwjirOTF35r-AhULHDQIHQXqChwQ2-cCegQIABAA&oq=paintings&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1C9DVi1EmDwE2gAcAB4AIABqAGIAeMIkgEDNS41mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=4JsxZOK8CYu40PEPhdSr4AE&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=cartoon&tbm=isch&ved=2ahUKEwid9c-W4Jr-AhVgADQIHf3LAdYQ2-cCegQIABAA&oq=cartoon&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIICAAQgAQQsQM6BAgjECc6BQgAEIAEUMgEWOwMYKMOaABwAHgAgAFviAHjBZIBAzcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=iZwxZN2MKuCA0PEP_ZeHsA0&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=animals&tbm=isch&ved=2ahUKEwixnMK34Jr-AhWKBzQIHdT6AE8Q2-cCegQIABAA&oq=animals&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyCwgAEIAEELEDEIMBMgUIABCABDIFCAAQgAQyCAgAEIAEELEDMgUIABCABDIICAAQgAQQsQMyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6CggAEIoFELEDEEM6BwgAEIoFEENQrAVY6gpg9wtoAHAAeACAAW2IAeMFkgEDNy4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=zpwxZLH9KIqP0PEP1PWD-AQ&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=drawings&tbm=isch&ved=2ahUKEwixnMK34Jr-AhWKBzQIHdT6AE8Q2-cCegQIABAA&oq=drawings&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzoECCMQJzoFCAAQgAQ6CAgAEIAEELEDUPYEWKEMYOwMaABwAHgAgAFyiAH1BZIBAzcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=zpwxZLH9KIqP0PEP1PWD-AQ&bih=609&biw=1263&hl=en',
        'https://www.google.com/search?q=people+doing+things+real&tbm=isch&ved=2ahUKEwjQ-9ST95r-AhXhGTQIHWqpDgEQ2-cCegQIABAA&oq=people+doing+things+real&gs_lcp=CgNpbWcQAzIGCAAQCBAeOgQIIxAnOgcIABCKBRBDOgUIABCABDoGCAAQBRAeUIEDWNcHYKQJaABwAHgAgAGoAYgBsQWSAQMyLjSYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=obQxZJCnHuGz0PEP6tK6CA&bih=609&biw=1280',
        'https://www.google.com/search?q=portraits&tbm=isch&chips=q:portraits,g_1:photography:G7hcdP5jhN0%3D&hl=en&sa=X&ved=2ahUKEwj9tra295r-AhW2GTQIHYkLApAQ4lYoAnoECAEQKg&biw=1263&bih=609',
        'https://www.google.com/search?q=watercolour&tbm=isch&ved=2ahUKEwiDrJ6v_5r-AhURAzQIHWRjBnIQ2-cCegQIABAA&oq=watercolour&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6BwgAEIoFEENQ5QZYjw9glhBoAHAAeACAAW-IAcQGkgEDNy4ymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=Pr0xZIOVMZGG0PEP5MaZkAc&bih=609&biw=1263&hl=en',
        'https://www.google.com/search?q=oil+painting&tbm=isch&ved=2ahUKEwi5l56y_5r-AhX6HDQIHQlkBZ4Q2-cCegQIABAA&oq=oi&gs_lcp=CgNpbWcQARgAMgoIABCKBRCxAxBDMgcIABCKBRBDMgcIABCKBRBDMgoIABCKBRCxAxBDMggIABCABBCxAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIICAAQgAQQsQMyBQgAEIAEOgQIIxAnOggIABCxAxCDAVDECljhC2CiF2gAcAB4AIABY4gBkAKSAQEzmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=Rb0xZPngBfq50PEPiciV8Ak&bih=609&biw=1263&hl=en'
       ]

# Categories for images
categories = [
    'a_graphic_deisgn_art', 'b_anime_art', 'c_people_outside', 'd_digital_art',
    'e_digital_art_portrait', 'f_igital_art_drawing', 'g_abstract_art', 'h_paintings', 'i_cartoon',
    'j_animals', 'k_drawings', 'l_people_doing_things','m_real_portraits', 'n_watercolour', 'o_oil'
]

def scroll_down(wd):
    """
    Scroll down the webpage using JavaScript to load more content.
    """
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(delay)

def download_images(url, category):
    """
    Download images from the given URL and save them in the specified category folder.
    """
    # Initialize the Chrome webdriver
    wd = webdriver.Chrome(executable_path=PATH)

    # Visit the URL
    wd.get(url)
    delay = 0

    skips = 0

    # Create the category folder if it doesn't exist
    category_folder = os.path.join(HUMAN, category)
    os.makedirs(category_folder, exist_ok=True)

    try:
        # Scroll and collect images
        while skips <= 50:
            # Find thumbnail elements on the webpage
            thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")
            time.sleep(3)

            # Iterate over the thumbnails and download images
            for img in thumbnails:
                src = img.get_attribute('src')
                if src not in image_urls:
                    image_urls.add(src)
                    try:
                        # Download the image and save it to the specified category folder
                        urllib.request.urlretrieve(src, os.path.join(category_folder, f'{len(image_urls)}.png'))
                    except:
                        # If download fails, continue with the next image
                        continue

            skips += 1

            # Scroll down to load more content
            scroll_down(wd)
    finally:
        # Quit the webdriver
        wd.quit()

# Create a set to store unique image URLs
image_urls = set()

# Iterate over the URLs and categories
for search in range(len(urls)):
    url = urls[search]
    category = categories[search]

    # Download images for the current URL and category
    download_images(url, category)


# Scrape DALLE images

This code is used to scrape images from the website dalle2.gallery, which hosts AI-generated images created by DALLE. It automates the process of visiting the website, scrolling down to load more images, and downloading the unique image URLs until the specified maximum number of images (max_images) is reached. The downloaded images are saved with sequential numbers as filenames.

In [ ]:
from multiprocessing.sharedctypes import Value
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image
import time
import os
import urllib

PATH = 'C:\\Users\\Karl Marie Yazigi\\Desktop\\chromedriver_win32\\chromedriver'

def scroll_down(wd):
    """
    Scroll down the webpage using JavaScript to load more content.
    """
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(delay)

# Initialize the Chrome webdriver
wd = webdriver.Chrome(executable_path=PATH)

url = 'https://dalle2.gallery/#search-random'
DALLE = 'C:\\Users\\Karl Marie Yazigi\\Desktop\\DALLE\\'

# Visit the URL
wd.get(url)
max_images = 50000
delay = 0
image_urls = set()
skips = 0
before = 0

while len(image_urls) < max_images:
    # Print the number of collected image URLs every 100 images
    if len(image_urls) % 100 == 0:
        print(len(image_urls))

    before = len(image_urls)

    # Find thumbnail elements on the webpage
    thumbnails = wd.find_elements(By.TAG_NAME, 'img')
    time.sleep(1)

    # Iterate over the thumbnails and download images
    for img in thumbnails[len(image_urls):]:
        try:
            src = img.get_attribute('src')
            if src not in image_urls:
                image_urls.add(src)
                try:
                    # Download the image and save it with a sequential number as the filename
                    urllib.request.urlretrieve(src, os.path.join(DALLE, f'{len(image_urls)}.png'))
                except:
                    # If download fails, continue with the next image
                    continue

        except:
            # If an exception occurs during image processing, continue with the next image
            continue

    # Scroll down to load more content
    scroll_down(wd)

# Quit the webdriver
wd.quit()
